In [2]:
# Import sales data
import pandas as pd

df = pd.read_csv('sales_dataset.csv')
df.describe()

,"The Villages, FL, 32162 (28.9269978, -82.0041758)"
count,5140
unique,5140
top,"Littleton, CO, 80123 (39.613321, -105.016649)"
freq,1


In [9]:
# Add weight as string and float
df['weight'] = df['product_name'].str.extract('((?:[kK][gG]\.?)[ ]*\d+(?:[.,]\d+)?|\d+(?:[.,]\d+)?[ ]*(?:[kK][gG]\.?))')
df.loc[df['product_name'] == 'Plastic Square Tub Lid 714 lids per box PPWLID', 'weight' ] = '60kg'
df.loc[df['product_name'] == 'Plasctic Square Tub 102 Tubs per box PPWTUB', 'weight' ] = '60kg'

df['kg'] = df['weight'].replace('kg','',regex=True).astype(float)


In [15]:
# Add long,lat data to each location
from geopy.geocoders import Nominatim
from geopy import distance
import time

d = distance.distance
g = Nominatim(user_agent="ne_location")
dalas = g.geocode('Dalas, TX')[1]

df['city_state'] = df['city']+ ', '+ df['state_id'] + ', ' + df['zipcode'].astype(str).str.zfill(5)
loc = df.city_state.drop_duplicates()
loc_dataset = pd.DataFrame({'city_state':loc, 'geo': loc})


loc_dataset = loc_dataset.reset_index()

# if this throws an error just run it again 
# api timeouts are expected
for index, row in loc_dataset.iterrows():
    if not isinstance(loc_dataset.iloc[index]['geo'], tuple):
        geo = g.geocode(row['city_state'], timeout=None)[1]
        loc_dataset.at[index, 'geo'] = geo
        print(f"{row['city_state']} {loc_dataset.at[index,'geo']}")
        time.sleep(1)




The Villages, FL, 32162 (28.9269978, -82.0041758)
Littleton, CO, 80123 (39.613321, -105.016649)
Minneapolis, MN, 55433 (44.9772995, -93.2654692)
Anniston, AL, 36201 (33.6490564, -85.8319698)
Greenwood, IN, 46143 (39.6136578, -86.1066526)
Boynton Beach, FL, 33437 (26.5256179, -80.0666026)
Paw Paw, MI, 49079 (42.2178171, -85.8911246)
Lancaster, SC, 29720 (34.730366624700686, -80.74926195202124)
Phoenix, AZ, 85009 (33.4484367, -112.074141)
Melbourne, FL, 32935 (28.117710000000002, -80.66295750056375)
Irving, TX, 75063 (32.862919500000004, -96.97917026548832)
Coppell, TX, 75019 (32.9552598, -97.0155703)
Magnolia, TX, 77354 (30.2094574, -95.7508098)
Toa Baja, PR, 00949 (18.4426255, -66.2545652)
Orange Park, FL, 32065 (30.1715755, -81.71362104591785)
Union City, CA, 94587 (37.5906286, -122.0173972)
Blytheville, AR, 72315 (35.9272953, -89.9189754)
Grayslake, IL, 60030 (42.3433518, -88.0412192)
Anoka, MN, 55303 (45.25159181717559, -93.4150551393058)
Westminster, SC, 29693 (34.6648197, -83.0965

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Santa+Maria%2C+CA%2C+93455&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [114]:
# Merge log,lat data to original table 
df_full = pd.merge(df, loc_dataset, on=['city_state','city_state'])
#Export data to csv file
df_full.to_csv("full_sales.csv", sep=',')